# PySpark Recipes: Chapter 5: GroupBys and Aggregations

GroupBy allows you to group rows together based off some column value. groupby() is an alias for groupBy(), and we have use them interchangeably so do not get confuse. While the methods are case sensitive, aliases of the methods when available, help you focus not on the upperCase or lowerCase of the letter.

In [1]:
import pyspark
from pyspark.sql import SparkSession


# creating a SparkSession object - you can change any of the configuration option you like. Remember this would
# get the existsing SparkSession and would not create a new one.
# So in case your previous notebook is still running - no issues
sparkSession = SparkSession \
                .builder \
                .master("local") \
                .appName("Pyspark Recipes - Importing Data") \
                .getOrCreate()

# Importing the data which we would use in our project
dfCensus = sparkSession.read.format('csv') \
            .options(header = True, inferSchema = True, sep = ",", enforceSchema = True,
                ignoreLeadingWhiteSpace = True, ignoreTrailingWhiteSpace = True) \
            .load('../datasets/charityml/censusdata.csv')

print(dfCensus.count())


45222


In [12]:
dfCensus.show(n=5)

+---+----------------+---------------+-------------+------------------+-----------------+-------------+-----+------+------------+------------+--------------+--------------+------+
|age|       workclass|education_level|education-num|    marital-status|       occupation| relationship| race|   sex|capital-gain|capital-loss|hours-per-week|native-country|income|
+---+----------------+---------------+-------------+------------------+-----------------+-------------+-----+------+------------+------------+--------------+--------------+------+
| 39|       State-gov|      Bachelors|         13.0|     Never-married|     Adm-clerical|Not-in-family|White|  Male|      2174.0|         0.0|          40.0| United-States| <=50K|
| 50|Self-emp-not-inc|      Bachelors|         13.0|Married-civ-spouse|  Exec-managerial|      Husband|White|  Male|         0.0|         0.0|          13.0| United-States| <=50K|
| 38|         Private|        HS-grad|          9.0|          Divorced|Handlers-cleaners|Not-in-fami

In [10]:
dfCensus.groupBy("marital-status").count().show()

+--------------------+-----+
|      marital-status|count|
+--------------------+-----+
|           Separated| 1411|
|       Never-married|14598|
|Married-spouse-ab...|  552|
|            Divorced| 6297|
|             Widowed| 1277|
|   Married-AF-spouse|   32|
|  Married-civ-spouse|21055|
+--------------------+-----+



We would be deconstructing the above sequence of steps to understand the mechanism happening behind the screen.

When you use the "groupby" method on a dataframe, the returning result is a GroupedData Object and not a dataframe. A GroupedData object is a transitional object, where the object is waiting for instruction on how to summarize the information contained in each group. 

Try doing a show on the following code and check the error for yourself. 

In [ ]:
objGroupByMaritalStatus = dfCensus.groupBy("marital-status")
objGroupByMaritalStatus.show()

While the above line of code would return an error, the following functions are completely valid: sum(), mean(), min(), max(), count(), avg(). It takes a column name in the object other than the one that is grouped by which are "non-string". 

Try calling one of the methods on the GroupedData obect, and see the response.

In [16]:
objGroupByMaritalStatus.max()

DataFrame[marital-status: string, max(age): int, max(education-num): double, max(capital-gain): double, max(capital-loss): double, max(hours-per-week): double]

When you use a "count()" method (but there is no show() method) on the object, we get a dataframe back containing the grouping column, in this case "marital-status". The resulting dataframe would contain only two columns, one on which was used for groupBy to create the GroupedData Object ("marital-status"), and the other on which the any of the aggregation operation was performed. 

No other "string" columns in the dataframe - dfCensus in this case - would be available, unless and until, they are used for a groupBy. 

In [18]:
dfGroupedByMaritalStatus = objGroupByMaritalStatus.count()
dfGroupedByMaritalStatus.show()

+--------------------+-----+
|      marital-status|count|
+--------------------+-----+
|           Separated| 1411|
|       Never-married|14598|
|Married-spouse-ab...|  552|
|            Divorced| 6297|
|             Widowed| 1277|
|   Married-AF-spouse|   32|
|  Married-civ-spouse|21055|
+--------------------+-----+



While the above code produced a "count" column, where have count of rows of the GroupedData objects, if you use any of the other aggregation functions - min, max, avg, sum, mean - unless and until you specify the column names, the method would be applied to all the non-string columns to the dataframe from which the GroupedData object was created, in our case dfCensus. 

In [80]:
dfGroupedByMaritalStatus = objGroupByMaritalStatus.max()
dfGroupedByMaritalStatus.show()

+--------------------+--------+------------------+-----------------+-----------------+-------------------+
|      marital-status|max(age)|max(education-num)|max(capital-gain)|max(capital-loss)|max(hours-per-week)|
+--------------------+--------+------------------+-----------------+-----------------+-------------------+
|           Separated|      90|              16.0|          99999.0|           3900.0|               99.0|
|       Never-married|      90|              16.0|          99999.0|           3770.0|               99.0|
|Married-spouse-ab...|      87|              16.0|          99999.0|           3004.0|               99.0|
|            Divorced|      90|              16.0|          99999.0|           3900.0|               99.0|
|             Widowed|      90|              16.0|          99999.0|           4356.0|               99.0|
|   Married-AF-spouse|      58|              16.0|          99999.0|           1485.0|               90.0|
|  Married-civ-spouse|      90|      

In [24]:
dfGroupedByMaritalStatus = objGroupByMaritalStatus.max("education-num", "capital-gain")
dfGroupedByMaritalStatus.show()

+--------------------+------------------+-----------------+
|      marital-status|max(education-num)|max(capital-gain)|
+--------------------+------------------+-----------------+
|           Separated|              16.0|          99999.0|
|       Never-married|              16.0|          99999.0|
|Married-spouse-ab...|              16.0|          99999.0|
|            Divorced|              16.0|          99999.0|
|             Widowed|              16.0|          99999.0|
|   Married-AF-spouse|              16.0|          99999.0|
|  Married-civ-spouse|              16.0|          99999.0|
+--------------------+------------------+-----------------+



Introducing the "agg" method. The generic "agg" method would call the aggregate across all rows in the dataframe column specified. It can take in arguments as a single column, or create multiple aggregation calls all at once using dictionary notaion, which we will see in a short while. 

The benefit of "agg" method, is that we can calculate many aggregations at a time on a single statement using Spark SQL aggregate functions sum(), avg(), min(), max() mean(), as you would see in a little while. 

In [5]:
dfCensus.groupby('marital-status').agg({'marital-status': 'count'}).show()

+--------------------+---------------------+
|      marital-status|count(marital-status)|
+--------------------+---------------------+
|           Separated|                 1411|
|       Never-married|                14598|
|Married-spouse-ab...|                  552|
|            Divorced|                 6297|
|             Widowed|                 1277|
|   Married-AF-spouse|                   32|
|  Married-civ-spouse|                21055|
+--------------------+---------------------+



When we call the "agg" method as a generic one, it can be applied to a dataframe without a "groupBy". Say for some reason you want to have a average of capital gain for the entire dataframe.

Not much useful, as mean, max, average are available in the describe() method, that we have seen in chapter 2.

In [27]:
dfCensus.agg({'capital-gain': 'avg'}).show()

+------------------+
| avg(capital-gain)|
+------------------+
|1101.4303436380524|
+------------------+



In [110]:
dfCensus.groupby('marital-status','education_level').agg({'income': 'count', 'capital-gain': 'max'}).show()

+--------------------+---------------+-------------+-----------------+
|      marital-status|education_level|count(income)|max(capital-gain)|
+--------------------+---------------+-------------+-----------------+
|             Widowed|   Some-college|          218|          15831.0|
|Married-spouse-ab...|   Some-college|          101|          27828.0|
|            Divorced|      Assoc-voc|          349|          15831.0|
|            Divorced|           11th|          172|          14084.0|
|            Divorced|           10th|          152|          14344.0|
|       Never-married|           10th|          438|          34095.0|
|  Married-civ-spouse|     Assoc-acdm|          665|          20051.0|
|           Separated|      Doctorate|           11|          14084.0|
|             Widowed|            9th|           28|           2062.0|
|            Divorced|        7th-8th|           84|           2964.0|
|           Separated|      Bachelors|          129|          99999.0|
|     

We had mentioned that the "agg" method accepts dictionary, so that you can specificy multiple columns. 

In [42]:
dicMulti = {
    "education-num" : "min",
    "education-num" : "max",
    "capital-gain" : "avg",
    "capital-loss" : "sum",
    "hours-per-week" : "min"
}

dfCensus.groupby('marital-status','education_level').agg(dicMulti).show()

+--------------------+---------------+-----------------+------------------+-------------------+------------------+
|      marital-status|education_level|sum(capital-loss)|max(education-num)|min(hours-per-week)| avg(capital-gain)|
+--------------------+---------------+-----------------+------------------+-------------------+------------------+
|             Widowed|   Some-college|          12312.0|              10.0|                1.0| 350.8256880733945|
|Married-spouse-ab...|   Some-college|           6208.0|              10.0|               15.0| 475.9207920792079|
|            Divorced|      Assoc-voc|          14690.0|              11.0|                3.0|  494.243553008596|
|            Divorced|           11th|           7954.0|               7.0|                1.0|213.94767441860466|
|            Divorced|           10th|           2974.0|               6.0|                1.0| 94.36842105263158|
|       Never-married|           10th|          24292.0|               6.0|     

One thing, you would notice in the above code, that two operations on the same column as a separate dictionary entry does not work. Of the "sum" and "avg" operation on the column "income", the latter "avg" replaces the former "sum", while producing the output dataframe. 

You cannot achieve the desired output via dictionaries. You would have to use sql functions available under sql package instead, as demonstrated in the code below. The below code also shows multiple aggregations being done using Spark SQL aggregate functions in a single line. 

In [43]:
from pyspark.sql import functions as F
dfCensus.groupby('marital-status','education_level').agg(
        F.min("education-num"),
        F.max("education-num"),
        F.avg("capital-gain"),
        F.sum("capital-loss"),
        F.min("hours-per-week")
        ).show()

+--------------------+---------------+------------------+------------------+------------------+-----------------+-------------------+
|      marital-status|education_level|min(education-num)|max(education-num)| avg(capital-gain)|sum(capital-loss)|min(hours-per-week)|
+--------------------+---------------+------------------+------------------+------------------+-----------------+-------------------+
|             Widowed|   Some-college|              10.0|              10.0| 350.8256880733945|          12312.0|                1.0|
|Married-spouse-ab...|   Some-college|              10.0|              10.0| 475.9207920792079|           6208.0|               15.0|
|            Divorced|      Assoc-voc|              11.0|              11.0|  494.243553008596|          14690.0|                3.0|
|            Divorced|           11th|               7.0|               7.0|213.94767441860466|           7954.0|                1.0|
|            Divorced|           10th|               6.0|     

The issue in the above examples, is that the output columns are in the form - aggregation_function(column_name), which is not intuitive for future dataframe operations. 

We would discuss three ways to rename a column with agg opertion in PySpark. But first the intuitive way of using the alias function, which would not work, as demonstrated below. It would not give an error though. The reason the below syntax does not work is because we are aliasing the whole dataframe instead of the column - max('education-num')

In [48]:
# The following code would work but would not rename the column. 
dfCensus.groupBy("marital-status").max('education-num').alias('max_education_num').show()

#max("education-num", "capital-gain")

+--------------------+------------------+
|      marital-status|max(education-num)|
+--------------------+------------------+
|           Separated|              16.0|
|       Never-married|              16.0|
|Married-spouse-ab...|              16.0|
|            Divorced|              16.0|
|             Widowed|              16.0|
|   Married-AF-spouse|              16.0|
|  Married-civ-spouse|              16.0|
+--------------------+------------------+



In [77]:
# Option 1
# Remember we have used alias F while importing from pyspark.sql import functions
# would not recommend - as you see, you have to select the all the column names, better would be Option 3.
dfCensus.groupBy("marital-status") \
         .avg('education-num') \
         .select(F.col("marital-status"),
                 F.col('avg(education-num)').alias('avg_education_num')) \
         .show()

+--------------------+------------------+
|      marital-status| avg_education_num|
+--------------------+------------------+
|           Separated| 9.369241672572644|
|       Never-married|10.017468146321415|
|Married-spouse-ab...| 9.289855072463768|
|            Divorced|10.096077497220898|
|             Widowed| 9.149569303054033|
|   Married-AF-spouse|              10.5|
|  Married-civ-spouse|10.325290904773214|
+--------------------+------------------+



In [82]:
# Option 2
# use "agg" method instead of calling the aggregation function directly
dfCensus.groupby('marital-status','education_level').agg(
        F.max("education-num").alias("max_education_num"),
        F.avg("capital-gain").alias("avg_capital_gain")
    ).show()

+--------------------+---------------+-----------------+------------------+
|      marital-status|education_level|max_education_num|  avg_capital_gain|
+--------------------+---------------+-----------------+------------------+
|             Widowed|   Some-college|             10.0| 350.8256880733945|
|Married-spouse-ab...|   Some-college|             10.0| 475.9207920792079|
|            Divorced|      Assoc-voc|             11.0|  494.243553008596|
|            Divorced|           11th|              7.0|213.94767441860466|
|            Divorced|           10th|              6.0| 94.36842105263158|
|       Never-married|           10th|              6.0|108.32420091324201|
|  Married-civ-spouse|     Assoc-acdm|             12.0| 864.8827067669173|
|           Separated|      Doctorate|             16.0|1494.3636363636363|
|             Widowed|            9th|              5.0| 73.64285714285714|
|            Divorced|        7th-8th|              4.0| 61.19047619047619|
|           

In [78]:
# Option 3
# using "withColumnRenamed"
dfCensus.groupBy("marital-status") \
         .avg('education-num') \
         .withColumnRenamed('avg(education-num)', 'avg_education_num') \
         .show()

+--------------------+------------------+
|      marital-status| avg_education_num|
+--------------------+------------------+
|           Separated| 9.369241672572644|
|       Never-married|10.017468146321415|
|Married-spouse-ab...| 9.289855072463768|
|            Divorced|10.096077497220898|
|             Widowed| 9.149569303054033|
|   Married-AF-spouse|              10.5|
|  Married-civ-spouse|10.325290904773214|
+--------------------+------------------+



Thoough Option 1 is covered for sake of completeness, we would recommend use of either Option 2 or Option 3.

We have seen introduction to PySpark aggregations and groupBys. The real power of PySpark lies in how these are use in combination with other functions of Spark. (we use functions and methods interchangeably)

For example, what if you want to have an aggregation based on a specific filter?

In [89]:
dfCensus.groupby('marital-status') \
        .agg({'capital-gain': 'avg'}) \
        .where(F.col('marital-status') == 'Never-married') \
        .show()



+--------------+-----------------+
|marital-status|avg(capital-gain)|
+--------------+-----------------+
| Never-married|402.4831483764899|
+--------------+-----------------+



Now that, you have applied filter on "marital-status", can you apply filter on "age"? No. 

Check out the error in the below code. 

In [ ]:
# You cannot appply filter on "age"
dfCensus.groupby('marital-status') \
        .agg({'capital-gain': 'avg'}) \
        .where(F.col('age') >= 24) \
        .show()

When you run the above code, you would get an error as follows - 

<span style="color:red;">AnalysisException: "cannot resolve '`age`' given input columns: [marital-status, avg(capital-gain)];;\n'Filter ('age >= 24)\n+- Aggregate [marital-status#14], [marital-status#14, avg(capital-gain#19) AS avg(capital-gain)#2563]\n   +- Relation[age#10,workclass#11,education_level#12,education-num#13,marital-status#14,occupation#15,relationship#16,race#17,sex#18,capital-gain#19,capital-loss#20,hours-per-week#21,native-country#22,income#23] csv\n"</span>

Here when we are using method chaining, it is not dfCensus which is available to all methods, but output of grouby() which is a GroupedData object containing the columns "marital-status" and "avg(capital-gain)" are passed to the where clause. 

Thus the columns specified in the groupBy and the agg, are available for filter operations. Read the above error, you cannot apply the filter on the column used for aggregation, but you can apply the filter on the aggregated column output as demonstrated below. 



In [93]:
# You cannot appply filter on "age" but only on columns returned by GroupedData object
dfCensus.groupby('marital-status') \
        .agg({'capital-gain': 'avg'}) \
        .where(F.col('avg(capital-gain)') >= 1000) \
        .show()

+------------------+------------------+
|    marital-status| avg(capital-gain)|
+------------------+------------------+
| Married-AF-spouse|        3353.03125|
|Married-civ-spouse|1736.5056281168368|
+------------------+------------------+



So what if you want to fire the groupBy on the subset. One of the approach is to apply filter() first and then pass the subset to the groupBy method.

In the next two lines of code, compare the output of both.

In [95]:
# You cannot appply filter on "age"
dfCensus.filter("age>=24") \
        .groupby('marital-status') \
        .agg({'capital-gain': 'avg'}) \
        .show()

+--------------------+------------------+
|      marital-status| avg(capital-gain)|
+--------------------+------------------+
|           Separated| 649.4770710059172|
|       Never-married| 557.2544247787611|
|Married-spouse-ab...| 708.4291262135922|
|            Divorced|  834.120875583454|
|             Widowed| 647.5765907305578|
|   Married-AF-spouse| 3973.962962962963|
|  Married-civ-spouse|1769.0337955779673|
+--------------------+------------------+



Another approach is to pass the filters in the aggregate clause, which even after applying groupBy the main dataframe is available for application of filters.

Check out the code below and compare it with the previous output - 

In [108]:
# While you cannot appply filter on "age" in the error code a few steps above, this works well perfectly
dfCensus.groupby('marital-status') \
        .agg(
            F.avg(
                F.when(F.col('age')>=24,
                F.col('capital-gain'))
            )).show()

+--------------------+------------------------------------------------+
|      marital-status|avg(CASE WHEN (age >= 24) THEN capital-gain END)|
+--------------------+------------------------------------------------+
|           Separated|                               649.4770710059172|
|       Never-married|                               557.2544247787611|
|Married-spouse-ab...|                               708.4291262135922|
|            Divorced|                                834.120875583454|
|             Widowed|                               647.5765907305578|
|   Married-AF-spouse|                               3973.962962962963|
|  Married-civ-spouse|                              1769.0337955779673|
+--------------------+------------------------------------------------+



In [109]:
# Just a sanity check to see the output without any filters
dfCensus.groupby('marital-status') \
        .agg({'capital-gain': 'avg'}) \
        .show()

+--------------------+------------------+
|      marital-status| avg(capital-gain)|
+--------------------+------------------+
|           Separated| 623.0673281360737|
|       Never-married| 402.4831483764899|
|Married-spouse-ab...| 660.9438405797101|
|            Divorced| 825.1003652532952|
|             Widowed| 645.5481597494127|
|   Married-AF-spouse|        3353.03125|
|  Married-civ-spouse|1736.5056281168368|
+--------------------+------------------+



So, which approach to use - using sql functions or the Spark methods. Depends on your comfort level, but cleaner and readable - the better. You got a strange column name when you used the "when" method (filter is not a method available in PySpark SQL package), we can use alias to rename the output as demonstrated below.

While we would be covering using all the methods in combination to perform complex operation, in the chapter - Bringing it together, where you would see many such examples, here is a little taste of what one can expect.

In [103]:
dfSomeOutput = dfCensus.groupby('marital-status') \
    .agg( \
            F.sum(  \
                F.when( \
                        F.trim(F.col('marital-status')).isin( \
                            ["Separated", "Divorced", "Widowed"] \
                        ), \
                        F.col("capital-gain"), 
                    ).otherwise(0) \
            ).alias("capital_gain_total"), \
            F.count("marital-status").alias("total_count"), \
        ) \
.withColumn( \
    "some_ratio", F.col("capital_gain_total") / F.col("total_count") \
) \

In [104]:
dfSomeOutput.show()

+--------------------+------------------+-----------+-----------------+
|      marital-status|capital_gain_total|total_count|       some_ratio|
+--------------------+------------------+-----------+-----------------+
|           Separated|          879148.0|       1411|623.0673281360737|
|       Never-married|               0.0|      14598|              0.0|
|Married-spouse-ab...|               0.0|        552|              0.0|
|            Divorced|         5195657.0|       6297|825.1003652532952|
|             Widowed|          824365.0|       1277|645.5481597494127|
|   Married-AF-spouse|               0.0|         32|              0.0|
|  Married-civ-spouse|               0.0|      21055|              0.0|
+--------------------+------------------+-----------+-----------------+



This conclude our recipe on GroupBys and aggregations, which we have covered here in more detail. However GroupBys and Joins causes reshuffle, and movement of the data across the nodes, and can impact your Spark code processing time. 

We would discuss the approaches in detail in our Chapter - Optimizing Spark. Till then, let us move to our next notebook. 